In [1]:
import os
from ae_functions import get_folder_pickle_files
from load_data import *
import matplotlib.pyplot as plt
from ae_functions import *
from torch.utils.data import DataLoader
import numpy as np
from acoustic_emission_dataset import AcousticEmissionDataset

# Load AE Dataset

In [2]:
# SIGNAL PROCESSING CONSTANTS
SIG_LEN = 1024           # [samples / signal] ;
DT = 10**-7              # [seconds] ; sample period / time between samples
LOW_PASS = 0      # [Hz] ; low frequency cutoff
HIGH_PASS = 100*10**4    # [Hz] ; high frequency cutoff
FFT_UNITS = 1000         # FFT outputs in Hz, this converts to kHz
NUM_BINS = 26            # For partial power

# FILE I/O
JSON_DATA_FILE = 'E:/file_cabinet/phd/projects/plb_dataset_supervised_learning/Data/220426_PLB_data.json'

# Load AE data
ae_dataset = AcousticEmissionDataset(JSON_DATA_FILE,SIG_LEN,DT,LOW_PASS,
                                     HIGH_PASS,FFT_UNITS,NUM_BINS,0,0)
total_count = len(ae_dataset)
test_percent = 1 - 0.8
train_count = int(0.8 * total_count)
test_count = total_count - train_count
torch.manual_seed(41) 
train, test = torch.utils.data.random_split(ae_dataset,
                                                  (train_count,
                                                    test_count))

angles = ae_dataset.angles # what the one hot encoded targets map to
num_classes = len(angles)  # how many diff angles, for model output dim
example_feature_vec, _ = ae_dataset[0] # to determine feature dim
feature_dim = example_feature_vec.shape[0] # for model creation input dim

print(f"Available angles: {angles}")

targets_one_hot is the one hot encoding for angle. ex: [1 0 1]

Shape of waves is: torch.Size([514, 1024])
Datatype of waves is: torch.float32
waves requires grad: False
Shape of targets is: torch.Size([514])
Datatype of targets is: torch.int64
targets requires grad: False
Ex: 0
Shape of targets_one_hot is: torch.Size([514, 6])
Datatype of targets_one_hot is: torch.float32
targets_one_hot requires grad: False
Ex: tensor([1, 0, 0, 0, 0, 0])

AcousticEmissionDataset loaded in!

Available angles: ['20deg' '22deg' '26deg' '30deg' '36deg' '40deg']


# Load Trained Model

In [38]:
pth_path = './expert-sweep-6.pth'
model_num = 1
feature_dim = 1024
model = load_model(pth_path, model_num, feature_dim,num_classes=6)
torch.load(pth_path)

NeuralNetwork_01(
  (layers): Sequential(
    (0): Linear(in_features=1024, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=6, bias=True)
  )
)


OrderedDict([('layers.0.weight',
              tensor([[-2.0924e-01, -1.9225e-01, -1.4448e-01,  ..., -3.4469e-01,
                       -2.9842e-01, -1.7765e-01],
                      [-2.4719e-02, -2.6589e-03, -3.3020e-02,  ...,  2.8793e-03,
                        3.2809e-02,  2.3550e-02],
                      [-3.2771e-02, -1.5035e-02, -5.6243e-03,  ..., -1.7003e-02,
                       -1.0280e-03, -1.8324e-02],
                      ...,
                      [ 5.4114e-01,  6.2828e-01,  8.4444e-01,  ...,  2.0335e-01,
                        1.9232e-01,  1.9416e-01],
                      [-9.2593e-01, -1.2193e+00, -1.1831e+00,  ..., -3.0139e-01,
                       -1.7542e-01, -4.2762e-02],
                      [-1.1876e-02,  2.5473e-02, -2.5260e-02,  ..., -2.3421e-03,
                        2.5627e-02, -1.0742e-02]])),
             ('layers.0.bias',
              tensor([ 0.2128,  0.0044, -0.0013,  0.1789,  0.0060,  0.0127, -0.0128, -0.3613,
                      -0.0

# Evaluated Trained Model on Waveforms of Specific Angle

In [10]:

def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

train_loader = make_loader(train, batch_size=10)
test_loader = make_loader(test, batch_size=10)

In [26]:
# Evaluate model accuracy on test data set

model.eval()

with torch.no_grad():
    
    test_predicted = []
    test_actual = []

    for inputs, targets in test_data_loader:
        
        prediction = model(inputs) # Tensor (1,3) -> ex. [0.1, 0.01,0.6]
        
        prediction = prediction[0].argmax()
        actual = targets[0].argmax()
        
        test_predicted.append(prediction) # ex. [0,0,1]
        test_actual.append(actual)
        
# Obnoxious data type converting
test_actual = torch.tensor(test_actual, device = 'cpu')
test_predicted = torch.tensor(test_predicted, device = 'cpu')

<Figure size 360x216 with 0 Axes>

In [35]:
from sklearn.metrics import classification_report
print(classification_report(test_predicted,test_actual,
                                target_names=angles))

              precision    recall  f1-score   support

       20deg       1.00      1.00      1.00        20
       22deg       0.93      1.00      0.97        14
       26deg       1.00      0.95      0.97        20
       30deg       1.00      0.94      0.97        18
       36deg       0.93      0.93      0.93        15
       40deg       0.94      1.00      0.97        16

    accuracy                           0.97       103
   macro avg       0.97      0.97      0.97       103
weighted avg       0.97      0.97      0.97       103



# Visualize Predictions

In [5]:
create_figure('', 1, 1, width=5, height=3,
                  suptitle_font_size=18, default_font_size=10,
                  title_font_size=22, axes_font_size=18, tick_font_size=20,
                  legend_font_size=14, w_space=0.25, h_space=0.25)

mean = test_predicted.mean()
std = test_predicted.std()
plt.hist(test_predicted,bins=30)
plt.axvline(test_predicted.mean(),color='red',linestyle='--',label=f'Mean: {mean:.3g}')
plt.axvline(mean+std,color='purple',linestyle='--',label=f'Std: {std:.3g}')
plt.axvline(mean-std,color='purple',linestyle='--')
plt.ylabel('Num Predicted')
plt.xlabel('Angle (deg)')
plt.xlim([15, 45])
plt.title(f'Model Predictions on {test_angles[0]}')
plt.legend()
plt.show()

NameError: name 'test_predicted' is not defined

<Figure size 360x216 with 0 Axes>